# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env #Package tested on Python 3.6.8
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you. 
```
$ python 
>>> import cosmicfish
>>> cosmicfish.install_class('<project-directory>/class')
>>> exit()
```

# Setup Fiducial Cosmology

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("/Volumes/SSD01/data/")
classpath = os.path.join(projectdir, "class")

Specify the values of your fiducial cosmology and other physical constants.

In [ ]:
#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 1, #Note the change! 
        "m_ncdm" : 0.1, #Comment out for N_ncdm=0
        "T_ncdm" : 1.4/2.726, #Comment out for N_ncdm=0
        "z_pk" : 0.7, 
        "T_cmb" : 2.726} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']
T_cmb = fid['T_cmb'] #Units of K

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc

Specify the parameter values you'd like to use to compute the forecast...

In [ ]:
#Setup redshift bins, fiducial relica masses to sample
z_table = np.arange(1.0, 1.1, 0.1)
m_chi = 0.1

# Convergence Analysis

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `relic_convergence_analysis` class of `cosmicfish`. 

We pass to `relic_convergence_analysis` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology to values specified by the user and compute the corresponding power spectrum derivatives. You can choose to pass specific values of the fiducial cosmology, or choose to vary relative to the fiducial cosmology. 

First, specify the points in parameter space you'd like to sample: 

In [ ]:
T_chi_table = np.arange(1.8, 2.1, 0.02) / T_cmb #Units of T_cmb. We specify absolute values for the T_chi parameter.
nonT_relvary = np.arange(1.01, 1.11, 0.01) #We specify values relative to the fiducial cosmology for other parameters.

Then, we generate instances of the `relic_convergence_analysis` class. It will search for pre-existing datasets for the specified points in parameter space in the datastore directory - otherwise, it will generate the dataset and place it in datastore. 

In [ ]:
conv1 = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore); 
conv2 = cf.relic_convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv3 = cf.relic_convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv4 = cf.relic_convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv5 = cf.relic_convergence_analysis(fid, 'h', 'rel', nonT_relvary, z_table, None, classpath, datastore);

At this point, all the useful information generated in the convergence analysis is accessed through the methods of the `relic_convergence_analysis` objects. We use those methods to plot the results of the analysis...

### Convergence Power Spectra

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv2.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv3.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv4.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv5.plot_ps(z_index=0, xscale='linear', plotdata=True)

### Convergence Derivatives of Power Spectra

In [ ]:
conv1.plot_dps(z_index=0, xscale='linear')
conv2.plot_dps(z_index=0, xscale='linear')
conv3.plot_dps(z_index=0, xscale='linear')
conv4.plot_dps(z_index=0, xscale='linear')
conv5.plot_dps(z_index=0, xscale='linear')

### Convergence Difference of Derivatives of Power Spectra

In [ ]:
conv1.plot_delta_dps(z_index=0, xscale='linear')
conv2.plot_delta_dps(z_index=0, xscale='linear')
conv3.plot_delta_dps(z_index=0, xscale='linear')
conv4.plot_delta_dps(z_index=0, xscale='linear')
conv5.plot_delta_dps(z_index=0, xscale='linear')

### Repeat for Other $m_\chi$

We can easily repeate the analysis above for other choice of $m_\chi$. 

In [ ]:
conv1a = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -1.0), classpath, datastore)
conv1b = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.8), classpath, datastore)
conv1c = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.6), classpath, datastore)
conv1d = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.4), classpath, datastore)
conv1e = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., -0.2), classpath, datastore)
conv1f = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.0), classpath, datastore)
conv1g = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.2), classpath, datastore)
conv1h = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.4), classpath, datastore)
conv1i = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.6), classpath, datastore)
conv1j = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 0.8), classpath, datastore)
conv1k = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, np.power(10., 1.0), classpath, datastore)

conv1a.plot_dps(z_index=1, xscale='linear')
conv1b.plot_dps(z_index=1, xscale='linear')
conv1c.plot_dps(z_index=0, xscale='linear')
conv1d.plot_dps(z_index=0, xscale='linear')
conv1e.plot_dps(z_index=0, xscale='linear')
conv1f.plot_dps(z_index=0, xscale='linear')
conv1g.plot_dps(z_index=0, xscale='linear')
conv1h.plot_dps(z_index=0, xscale='linear')
conv1i.plot_dps(z_index=0, xscale='linear')
conv1j.plot_dps(z_index=0, xscale='linear')
conv1k.plot_dps(z_index=0, xscale='linear')

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.
# Analysis - No Commentary

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cosmicfish as cf


#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("/Volumes/SSD01/data/")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 1, #Note the change! 
        "z_pk" : 0.7, 
        "T_cmb" : 2.726} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']
T_cmb = fid['T_cmb'] #Units of K

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc


#Setup redshift bins, fiducial relica masses, relic temps to sample
#z_table = np.arange(1.4, 1.5, 0.1)
z_table = np.array([1.4])
m_chi = 1.0
#T_chi_table = np.arange(1.0, 2.0, 0.1) / T_cmb #Units of T_cmb
T_chi_table = np.array([1.5]) / T_cmb #Units of T_cmb
nonT_relvary = np.arange(1.01, 1.11, 0.01)

In [ ]:
conv1 = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.01)
conv2 = cf.relic_convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
conv3 = cf.relic_convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
conv4 = cf.relic_convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01);
conv5 = cf.relic_convergence_analysis(fid, 'h', 'rel', nonT_relvary, z_table, None, classpath, datastore, dstep=0.01); 

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv2.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv3.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv4.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv5.plot_ps(z_index=0, xscale='linear', plotdata=True)

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_dps(z_index=0, xscale='linear')
conv2.plot_dps(z_index=0, xscale='linear')
conv3.plot_dps(z_index=0, xscale='linear')
conv4.plot_dps(z_index=0, xscale='linear')
conv5.plot_dps(z_index=0, xscale='linear')

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_delta_dps(z_index=0, xscale='linear')
conv2.plot_delta_dps(z_index=0, xscale='linear')
conv3.plot_delta_dps(z_index=0, xscale='linear')
conv4.plot_delta_dps(z_index=0, xscale='linear')
conv5.plot_delta_dps(z_index=0, xscale='linear')

In [ ]:
#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(1.0, 1.1, 0.1)
m_chi = 0.1
T_chi_table = np.arange(1.5, 1.6, 0.1) / T_cmb #Units of T_cmb

conv1a = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.01)
conv1b = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.02)
conv1c = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.03)
conv1d = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.04)
conv1e = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.05)
conv1f = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.06)
conv1g = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.07)
conv1h = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.08)
conv1i = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.09)
conv1j = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore, dstep=0.10)

In [ ]:
sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0], label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0], label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0], label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0], label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0], label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0], label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0], label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0], label='step=8%')
ax1.plot(conv1i.spectra[0][0].k_table, conv1i.dps[0][0], label='step=9%')
ax1.plot(conv1j.spectra[0][0].k_table, conv1j.dps[0][0], label='step=10%')

ax1.set_title(r'$\partial P_g / \partial$ T_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of T_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0], label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0], label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0], label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0], label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0], label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0], label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0], label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0], label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0], label='step=9%')
ax2.plot(conv1j.spectra[0][0].k_table, conv1j.dlogps[0][0], label='step=10%')
ax2.set_title(r'$\partial log(P_g) / \partial$ T_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of T_ncdm)]')
ax2.legend()
        
plt.show()


domega_chi_dT_chi = (3* np.power(1.5, 2.) * 0.1) / (np.power(1.95, 3.) * 94.)

sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0]/domega_chi_dT_chi, label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0]/domega_chi_dT_chi, label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0]/domega_chi_dT_chi, label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0]/domega_chi_dT_chi, label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0]/domega_chi_dT_chi, label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0]/domega_chi_dT_chi, label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0]/domega_chi_dT_chi, label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0]/domega_chi_dT_chi, label='step=8%')
ax1.plot(conv1i.spectra[0][0].k_table, conv1i.dps[0][0]/domega_chi_dT_chi, label='step=9%')
ax1.plot(conv1j.spectra[0][0].k_table, conv1j.dps[0][0]/domega_chi_dT_chi, label='step=10%')

ax1.set_title(r'$\partial P_g / \partial$ omega_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0]/domega_chi_dT_chi, label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0]/domega_chi_dT_chi, label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0]/domega_chi_dT_chi, label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0]/domega_chi_dT_chi, label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0]/domega_chi_dT_chi, label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0]/domega_chi_dT_chi, label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0]/domega_chi_dT_chi, label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0]/domega_chi_dT_chi, label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0]/domega_chi_dT_chi, label='step=9%')
ax2.plot(conv1j.spectra[0][0].k_table, conv1j.dlogps[0][0]/domega_chi_dT_chi, label='step=10%')
ax2.set_title(r'$\partial log(P_g) / \partial$ omega_ncdm for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax2.legend()
        
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=10)
plt.figure(figsize=(15, 7.5))
ax1 = plt.subplot(1, 2, 1)

ax1.plot(conv1a.spectra[0][0].k_table, conv1a.dps[0][0]/domega_chi_dT_chi - conv1b.dps[0][0]/domega_chi_dT_chi, label='step=1%')
ax1.plot(conv1b.spectra[0][0].k_table, conv1b.dps[0][0]/domega_chi_dT_chi - conv1c.dps[0][0]/domega_chi_dT_chi, label='step=2%')
ax1.plot(conv1c.spectra[0][0].k_table, conv1c.dps[0][0]/domega_chi_dT_chi - conv1d.dps[0][0]/domega_chi_dT_chi, label='step=3%')
ax1.plot(conv1d.spectra[0][0].k_table, conv1d.dps[0][0]/domega_chi_dT_chi - conv1e.dps[0][0]/domega_chi_dT_chi, label='step=4%')
ax1.plot(conv1e.spectra[0][0].k_table, conv1e.dps[0][0]/domega_chi_dT_chi - conv1f.dps[0][0]/domega_chi_dT_chi, label='step=5%')
ax1.plot(conv1f.spectra[0][0].k_table, conv1f.dps[0][0]/domega_chi_dT_chi - conv1g.dps[0][0]/domega_chi_dT_chi, label='step=6%')
ax1.plot(conv1g.spectra[0][0].k_table, conv1g.dps[0][0]/domega_chi_dT_chi - conv1h.dps[0][0]/domega_chi_dT_chi, label='step=7%')
ax1.plot(conv1h.spectra[0][0].k_table, conv1h.dps[0][0]/domega_chi_dT_chi - conv1i.dps[0][0]/domega_chi_dT_chi, label='step=8%')

ax1.set_title(r'$\Delta(\partial P_g / \partial$ omega_ncdm) for z=1.0, m_ncdm=0.1 [eV]')
ax1.set_xlabel(r'k [Mpc$^{-1}$]')
ax1.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax1.legend()

ax2 = plt.subplot(1, 2, 2)
ax2.plot(conv1a.spectra[0][0].k_table, conv1a.dlogps[0][0]/domega_chi_dT_chi - conv1b.dlogps[0][0]/domega_chi_dT_chi, label='step=1%')
ax2.plot(conv1b.spectra[0][0].k_table, conv1b.dlogps[0][0]/domega_chi_dT_chi - conv1c.dlogps[0][0]/domega_chi_dT_chi, label='step=2%')
ax2.plot(conv1c.spectra[0][0].k_table, conv1c.dlogps[0][0]/domega_chi_dT_chi - conv1d.dlogps[0][0]/domega_chi_dT_chi, label='step=3%')
ax2.plot(conv1d.spectra[0][0].k_table, conv1d.dlogps[0][0]/domega_chi_dT_chi - conv1e.dlogps[0][0]/domega_chi_dT_chi, label='step=4%')
ax2.plot(conv1e.spectra[0][0].k_table, conv1e.dlogps[0][0]/domega_chi_dT_chi - conv1f.dlogps[0][0]/domega_chi_dT_chi, label='step=5%')
ax2.plot(conv1f.spectra[0][0].k_table, conv1f.dlogps[0][0]/domega_chi_dT_chi - conv1g.dlogps[0][0]/domega_chi_dT_chi, label='step=6%')
ax2.plot(conv1g.spectra[0][0].k_table, conv1g.dlogps[0][0]/domega_chi_dT_chi - conv1h.dlogps[0][0]/domega_chi_dT_chi, label='step=7%')
ax2.plot(conv1h.spectra[0][0].k_table, conv1h.dlogps[0][0]/domega_chi_dT_chi - conv1i.dlogps[0][0]/domega_chi_dT_chi, label='step=8%')
ax2.plot(conv1i.spectra[0][0].k_table, conv1i.dlogps[0][0]/domega_chi_dT_chi - conv1j.dlogps[0][0]/domega_chi_dT_chi, label='step=9%')
ax2.set_title(r'$\Delta(\partial log(P_g) / \partial$ omega_ncdm) for z=1.0, m_ncdm=0.1 [eV]')
ax2.set_xlabel(r'k [Mpc$^{-1}$]')
ax2.set_ylabel(r'[Mpc$^3$ / (units of omega_ncdm)]')
ax2.legend()
        
plt.show()




.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

In [1]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cosmicfish as cf


#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("~/Desktop/test/data/")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
       "n_s" : 0.96659,
       "omega_b" : 0.02226,
       "omega_cdm" : 0.11271,
       "tau_reio" : 0.059888,
       "h" : 0.70148,
       "z_pk" : 1.4, 
       "T_cmb" : 2.726,
       "N_ncdm" : 1., 
       "T_ncdm" : 1.5 / 2.726, 
       "m_ncdm" : 0.5} 


c = 2.9979e8 #Units of m/s
H = 1000. * 100. * fid['h'] #Units of m/s/Mpc
kp = 0.05 * fid['h'] #Units of 1/Mpc


#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(1.4, 2.0, 0.1)
ndensity = [0.63, 0.96, 0.91, 0.84, 0.76, 0.67, 0.60, 0.50, 0.40, 0.29, 0.19, \
0.18, 0.10, 0.06]
dstep = 0.03 #Vary each parameter by +-3% to calculate derivatives about fiducial 
mu_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

In [2]:
lightrelic = cf.relic_forecast(fid, z_table, ndensity, dstep, classpath, datastore)

Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872162.6250710.794513
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872162.99290010.161646
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872163.3723950.413440
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872163.74388720.786116
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872164.124990.025846
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872164.4939430.831994
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872164.86188480.927206
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872165.2320810.520807
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872165.59576490.383763
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872165.95749880.008158
Dataset already exists at: /Users/nicholasdepo

Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872196.6026380.949132
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872197.02744910.826719
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872197.4224440.634161
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872197.82810120.608743
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872198.2264530.610665
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872198.6223910.035568
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872199.00917980.961858
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872199.3930650.825363
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872199.7866240.051768
Dataset already exists at: /Users/nicholasdeporzio/Desktop/test/data/1561872200.18507220.722139
Dataset already exists at: /Users/nicholasdepo

In [3]:
lightrelic.gen_fisher(mu_step)

/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:187: RuntimeWarning: divide by zero encountered in log
  for zval in self.z_steps])
/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:190: RuntimeWarning: divide by zero encountered in log
  for zval in self.z_steps])
/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:191: RuntimeWarning: invalid value encountered in subtract
  diff = high - low
/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:535: RuntimeWarning: divide by zero encountered in log
  logP_mu_high = np.log(self.Pg) + np.log(self.RSD) + np.log(self.FOG)
/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:540: RuntimeWarning: divide by zero encountered in log
  logP_mu_low = np.log(self.Pg) + np.log(self.RSD) + np.log(self.FOG)
/Users/nicholasdeporzio/Doc

Fisher element ( 0 ,  0 ) calculated...
Fisher element ( 0 ,  1 ) calculated...
Fisher element ( 0 ,  2 ) calculated...
Fisher element ( 0 ,  3 ) calculated...
Fisher element ( 0 ,  4 ) calculated...
Fisher element ( 0 ,  5 ) calculated...
Fisher element ( 0 ,  6 ) calculated...
Fisher element ( 1 ,  0 ) calculated...
Fisher element ( 1 ,  1 ) calculated...


/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:701: RuntimeWarning: overflow encountered in multiply
  * (1. / self.k_table[kidx]))
/Users/nicholasdeporzio/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:721: RuntimeWarning: invalid value encountered in double_scalars
  * (self.k_table[kidx+1]-self.k_table[kidx]))


Fisher element ( 1 ,  2 ) calculated...
Fisher element ( 1 ,  3 ) calculated...
Fisher element ( 1 ,  4 ) calculated...
Fisher element ( 1 ,  5 ) calculated...
Fisher element ( 1 ,  6 ) calculated...
Fisher element ( 2 ,  0 ) calculated...
Fisher element ( 2 ,  1 ) calculated...
Fisher element ( 2 ,  2 ) calculated...
Fisher element ( 2 ,  3 ) calculated...
Fisher element ( 2 ,  4 ) calculated...
Fisher element ( 2 ,  5 ) calculated...
Fisher element ( 2 ,  6 ) calculated...
Fisher element ( 3 ,  0 ) calculated...
Fisher element ( 3 ,  1 ) calculated...
Fisher element ( 3 ,  2 ) calculated...
Fisher element ( 3 ,  3 ) calculated...
Fisher element ( 3 ,  4 ) calculated...
Fisher element ( 3 ,  5 ) calculated...
Fisher element ( 3 ,  6 ) calculated...
Fisher element ( 4 ,  0 ) calculated...
Fisher element ( 4 ,  1 ) calculated...
Fisher element ( 4 ,  2 ) calculated...
Fisher element ( 4 ,  3 ) calculated...
Fisher element ( 4 ,  4 ) calculated...
Fisher element ( 4 ,  5 ) calculated...


In [4]:
fisher = np.nan_to_num(lightrelic.fisher)
pd.DataFrame(fisher)

,0,1,2,3,4,5,6
0,5.371729e+29,8.567143e+20,0.000000e+00,0.000000e+00,1.797693e+308,-2.053279e+18,0.000000e+00
1,8.567143e+20,1.662430e+12,0.000000e+00,0.000000e+00,0.000000e+00,-3.277530e+09,0.000000e+00
2,0.000000e+00,0.000000e+00,1.797693e+308,1.797693e+308,0.000000e+00,0.000000e+00,1.797693e+308
3,0.000000e+00,0.000000e+00,1.797693e+308,1.797693e+308,0.000000e+00,0.000000e+00,1.797693e+308
4,1.797693e+308,0.000000e+00,0.000000e+00,0.000000e+00,1.797693e+308,-1.797693e+308,0.000000e+00
5,-2.053279e+18,-3.277530e+09,0.000000e+00,0.000000e+00,-1.797693e+308,7.849490e+06,0.000000e+00
6,0.000000e+00,0.000000e+00,1.797693e+308,1.797693e+308,0.000000e+00,0.000000e+00,1.797693e+308


In [5]:
covariance = np.linalg.inv(fisher)
pd.DataFrame(covariance)

LinAlgError: Singular matrix

In [6]:
print("Fully Marginalized Error on A_s: ", np.sqrt(covariance[0][0]))
print("Fully Marginalized Error on n_s: ", np.sqrt(covariance[1][1]))
print("Fully Marginalized Error on omega_b: ", np.sqrt(covariance[2][2]))
print("Fully Marginalized Error on oemga_cdm: ", np.sqrt(covariance[3][3]))
print("Fully Marginalized Error on h: ", np.sqrt(covariance[4][4]))
print("Fully Marginalized Error on tau_reio: ", np.sqrt(covariance[5][5]))
print("Fully Marginalized Error on omega_ncdm: ", np.sqrt(covariance[6][6]))

Fully Marginalized Error on A_s:  6.812131749646603e-13
Fully Marginalized Error on n_s:  7.26169420880119e-05
Fully Marginalized Error on omega_b:  2.6074297260705823e-06
Fully Marginalized Error on oemga_cdm:  5.788754035575009e-06
Fully Marginalized Error on h:  nan
Fully Marginalized Error on tau_reio:  0.029835813127793067
Fully Marginalized Error on omega_ncdm:  1.9771199957609997e-07


/Users/nicholasdeporzio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sqrt
  """


In [13]:
np.log([x for x in range(10)]) - np.log([1 + y/2. for y in range(10)])

/Users/nicholasdeporzio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/Users/nicholasdeporzio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in subtract
  """Entry point for launching an IPython kernel.


array([       nan, 0.69314718, 0.69314718, 0.69314718, 0.69314718,
       0.69314718, 0.69314718, 0.69314718, 0.69314718, 0.69314718])

In [8]:
np.array([[1,2], [3,4], [5,6]]) * (2 * np.array([[1,2], [3,4], [5,6]]))

array([[ 2,  8],
       [18, 32],
       [50, 72]])

In [15]:
type(np.log([1 + y/2. for y in range(10)]))

numpy.ndarray

In [6]:
np.shape([[1,2,3], [4,5,6]])

(2, 3)